In [1]:
import numpy as np
import pandas as pd
import logging
import os
from datetime import timedelta
from pathlib import Path

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.sandbox.client import SandboxClient
from tinkoff.invest.utils import now
from tinkoff.invest.caching.market_data_cache.cache import MarketDataCache
from tinkoff.invest.caching.market_data_cache.cache_settings import (
    MarketDataCacheSettings,
)

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import sys
sys.path.append("..") 
from Portfolio.portfolio_tools import *
from tp_config import TINK_DATA
import tink_port as tink

/home/samsmu/anaconda3/envs/t/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/samsmu/anaconda3/envs/t/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Вводим токен, получаем список доступных аккаунтов

In [2]:
BASE = 't.UFRJ8SC9hafVOhFxEUY7yf1wZ1gGhwJp-WCp9o4rnEChHWns0c3jQ21eQwoOW_RurFqeZpss2scJkmMQnomJ9g'
ETF = 't.6nHltT1dYSfrVTIV9zF72fxDlB2sXJbRD6iJNpZXTFAN61rmD7m71xPp9ko12ta1JxA06em4YdN36xicnBmjWg'
MOMENTUM = 't.24WV5_MMG1bQArK1WPp1_DYWD52f-VfGjpR1ci5Pqf0PJ948zWhDstoO_6d4wXIhFTMVsVJSgOzPElUIPEO4Mw'
SANDBOX = 't.qTfMeDk8iM5GLjIGj5Q5DVSnGdvOmSOzG4r3jQqdkdE2YUJMtFvBNb4v-Tyr50-4rxPBqia2jT-kBsE4NtoiKw'

token = SANDBOX
if token == SANDBOX:
    WorkClient = SandboxClient
else:
    WorkClient = Client

In [3]:
sess = tink.TinkSession(WorkClient, token)

accs = sess.get_accounts()
print("Количество аккаунтов:", len(accs.accounts))

print(accs.accounts[0].name)
account_id = accs.accounts[0].id

Количество аккаунтов: 1



#### База идентификаторов Tinkoff

In [5]:
port = sess.get_portfolio()
base = tink.get_id_base(token)
df_port = tink.port_to_df(port, base)

dfx = base[base["type"] == "shares"]
dfx = dfx[dfx["cur"] == "rub"]
base_ru = dfx.copy()

In [6]:
df_port['sums'] = df_port.quantity * df_port.price
df_port

,figi,ticker,name,quantity,lot_quantity,price,sums
0,RUB000UTSTOM,0-RUB,None,3066065,3066065,0.00,0.0
3,BBG004S68758,BANE,Башнефть,38,38,3437.50,130625.0
8,BBG004S686N0,BANEP,Башнефть - привилегированные акции,24,24,2339.00,56136.0
12,BBG00475K6C3,CHMF,Северсталь,29,29,1835.20,53220.8
7,BBG012YQ6P43,CIAN,Циан АДР,90,90,891.00,80190.0
1,BBG004731032,LKOH,ЛУКОЙЛ,5,5,7920.50,39602.5
5,BBG004S681B4,NLMK,НЛМК,200,20,229.70,45940.0
6,BBG00Y91R9T3,OZON,Ozon Holdings PLC,22,22,3831.50,84293.0
10,TCS00A103X66,POSI,Positive Technologies,35,35,2875.60,100646.0
2,BBG004S682Z6,RTKM,Ростелеком,920,92,99.00,91080.0


### Portfolio Black-Litterman Model

In [ ]:
mcaps = {}
for t in dfp.columns:
   # mcaps[t] = stock.info["marketCap"]
    mcaps[t] = 1
mcaps

In [ ]:
mcaps['SBER'] = 1.5
mcaps['POSI'] = 1.5

In [ ]:
imoex_figi = 'BBG333333333'
market_prices = tink.get_candles(token, imoex_figi, CandleInterval.CANDLE_INTERVAL_DAY)
market_prices =  tink.get_open_price(market_prices)

In [ ]:
dfp

In [ ]:
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel

S = risk_models.CovarianceShrinkage(dfp).ledoit_wolf()
delta = black_litterman.market_implied_risk_aversion(market_prices['ticker'])
delta

In [ ]:
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)
market_prior

In [ ]:
# You don't have to provide views on all the assets
viewdict = {
    "POSI": 0.10,
}

confidences = [
    0.6
]
bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict)

In [ ]:
intervals = [
    (0.1, 0.4),
]

variances = []
for lb, ub in intervals:
    sigma = (ub - lb)/2
    variances.append(sigma ** 2)

print(variances)
omega = np.diag(variances)

In [ ]:
# We are using the shortcut to automatically compute market-implied prior
bl = BlackLittermanModel(S, pi="market", market_caps=mcaps, risk_aversion=delta,
                        absolute_views=viewdict, omega=omega)

In [ ]:
# Posterior estimate of returns
ret_bl = bl.bl_returns()
ret_bl

In [ ]:
rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(viewdict)], 
             index=["Prior", "Posterior", "Views"]).T
rets_df

In [ ]:
S_bl = bl.bl_cov()


In [ ]:
from pypfopt import EfficientFrontier, objective_functions

ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
weights
     

In [ ]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(weights, dfp.iloc[-1], total_portfolio_value=100000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
alloc

### Model

## Load all shares

In [ ]:
from dataload import ReadData

data_reader = ReadData(ETF)
data_reader.read_id_base()
data_reader.read_candles(days = 50, verbose = False)
#data_reader.to_df()
#data_reader.save('portfolio_prices.csv')

In [6]:
prices_path = Path(TINK_DATA, 'Cache', 'portfolio_prices.csv')
with open(prices_path) as f:
    df_full = pd.read_csv(prices_path, index_col='date')
df_full

,ABIO,ABRD,AFKS,AFLT,AGRO,AKRN,ALRS,AMEZ,APTK,AQUA,...,VEON-RX,VKCO,VRSB,VSMO,VTBR,WUSH,YAKG,YNDX,ZAYM,ZILLP
date,,,,,,,,,,,,,,,,,,,,,
2024-02-19,104.40,273.8,18.150,38.61,1422.0,18038.0,68.78,71.00,15.050,884.0,...,42.95,660.0,699.0,39260.0,0.024360,263.18,103.60,3424.8,NaN,2712.01
2024-02-20,105.98,271.0,17.893,38.89,1400.0,18096.0,69.61,72.00,15.350,892.0,...,41.90,660.4,698.0,39340.0,0.024975,264.00,100.10,3311.0,NaN,2800.00
2024-02-21,105.00,261.0,17.500,37.74,1364.0,17984.0,69.81,69.32,15.030,870.0,...,40.55,644.0,660.5,38200.0,0.023600,255.00,97.45,3314.6,NaN,2615.03
2024-02-22,101.64,254.4,17.601,36.71,1346.0,17904.0,69.80,67.60,14.324,864.0,...,40.00,642.0,632.5,36840.0,0.023135,252.11,95.40,3255.0,NaN,2721.00
2024-02-26,101.96,261.0,18.000,37.60,1362.2,17962.0,70.89,69.86,14.332,880.0,...,43.00,653.0,656.0,37020.0,0.023850,258.01,95.40,3345.0,NaN,2819.99
2024-02-27,102.78,262.0,18.147,38.31,1418.0,18100.0,71.40,70.16,14.598,884.5,...,41.95,660.0,677.0,38800.0,0.023610,268.00,98.45,3415.0,NaN,2815.00
2024-02-28,104.04,266.2,18.029,37.72,1451.0,17952.0,72.85,69.72,14.574,879.5,...,42.55,651.2,680.5,37640.0,0.023400,266.90,97.65,3365.8,NaN,2754.00
2024-02-29,104.00,271.0,18.031,37.61,1423.4,17924.0,70.70,70.50,14.650,905.0,...,40.60,642.0,671.5,37320.0,0.023325,266.81,98.95,3384.0,NaN,2781.00
2024-03-01,106.08,265.4,18.500,38.56,1412.4,18050.0,71.12,70.30,14.636,906.0,...,40.60,665.0,670.0,38260.0,0.023430,268.00,97.75,3386.0,NaN,2813.93


#### Drops Momentum

In [7]:
dfp = df_full.copy()
drops = ["GTRK", "NTZL", "LSRG", 'TGKBP', 'FIVE', "VEON-RX"]
columns = [x for x in dfp.columns if x not in drops]
dfp = dfp[columns]

#### Drops Base

In [ ]:
dfp = df_full.copy()
index_assets = pd.read_csv('index_assets.csv')['asset'].values.tolist()
columns = [x for x in dfp.columns if x in index_assets]

drops = ['FIVE']
columns = [x for x in dfp.columns if x not in drops]

dfp = dfp[columns]

### RiskFolio

In [9]:
import riskfolio as rp

rms= ['MV', 'MSV',  'CVaR']

lookback = 30
df_period = dfp[-lookback:]

def riskfolio_weights(df_period, rm , obj):
    """
        obj - Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    """
    Y = df_period.pct_change().dropna()

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
#    port.solvers = ['MOSEK']
    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    # First we need to delete the cardinality constraint
    port.card = None 

    # Then we need to set the constraint on the minimum number of effective assets
    port.nea = 10
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = w[w.weights > 0.01]
    return w

#print_data(dfp)
df_period = dfp.dropna(axis = 1)

dfw = riskfolio_weights(df_period, 'CVaR', 'MaxRet')
dfw

You must convert self.cov to a positive definite matrix


,weights
AFKS,0.057568613
AFLT,0.043617713
ASTR,0.016019446
CIAN,0.019070299
DELI,0.028174497
HHRU,0.033813004
LENT,0.036272186
MOEX,0.011629519
OZON,0.029470950
PLZL,0.030558787


In [10]:
df_port['sums'] = df_port.quantity * df_port.price
sum_to_allocate = df_port.sums.sum() - 500

dfx = final_sums(dfw, sum_to_allocate, 50)
dfx['lot'] = 1
inds = dfx.index.values.tolist()

x = base_ru[base_ru['ticker'].isin (inds)]
s = x[['ticker', 'lot']].set_index('ticker')
dfx['lot'] = s
dfx = dfx.sort_values("weights", ascending = False)

prices = dfp.iloc[-1].T.loc[dfx.index]
dfx["price"] = prices

In [11]:
dfx["lot_quantity"] = np.round(dfx.weights/ (dfx.price * dfx.lot)).astype(int)
dfx["quantity"] =  dfx.lot * dfx.lot_quantity
dfx["sum"]= dfx.price * dfx.lot_quantity * dfx.lot
dfx = dfx.sort_values("weights", ascending = False)
dfx.to_csv("t.csv")
dfx = dfx.reset_index()
dfx.columns = ['ticker', 'weights', 'lot', 'price', 'lot_quantity','quantity', 'sums']
dfx = dfx[dfx.lot_quantity > 0]
dfx

,ticker,weights,lot,price,lot_quantity,quantity,sums
0,RBCM,21652.9,100,15.710,14,1400,21994.00
1,SFIN,20213.4,1,1800.000,11,11,19800.00
2,RNFT,13339.4,1,241.500,55,55,13282.50
3,AFKS,6511.3,100,24.180,3,300,7254.00
5,AFLT,4933.4,10,49.620,10,100,4962.00
6,LENT,4102.6,1,965.000,4,4,3860.00
7,HHRU,3824.4,1,4500.000,1,1,4500.00
8,POSI,3502.0,1,2837.000,1,1,2837.00
9,UGLD,3462.4,1000,0.953,4,4000,3812.00
11,OZON,3333.3,1,3881.000,1,1,3881.00


In [12]:
res_sum = sum_to_allocate - dfx.sums.sum()
print(res_sum)
res = []
for ind, row in dfx.iterrows():
    qty = res_sum/ row.price
    lot_qty = np.round(qty / row.lot).astype(int)
    pos_sum = lot_qty * row.lot * row.price
    print(row.ticker, lot_qty, res_sum, pos_sum)    
    if res_sum >= pos_sum:
        res.append(lot_qty)
        res_sum = res_sum -  pos_sum
    else:
        res.append(0)

9271.850000000006
RBCM 6 9271.850000000006 9426.0
SFIN 5 9271.850000000006 9000.0
RNFT 1 271.8500000000058 241.5
AFKS 0 30.35000000000582 0.0
AFLT 0 30.35000000000582 0.0
LENT 0 30.35000000000582 0.0
HHRU 0 30.35000000000582 0.0
POSI 0 30.35000000000582 0.0
UGLD 0 30.35000000000582 0.0
OZON 0 30.35000000000582 0.0
DELI 0 30.35000000000582 0.0
WUSH 0 30.35000000000582 0.0
SELG 0 30.35000000000582 0.0
CIAN 0 30.35000000000582 0.0
ASTR 0 30.35000000000582 0.0
RTKMP 0 30.35000000000582 0.0
MOEX 0 30.35000000000582 0.0
RTKM 0 30.35000000000582 0.0


In [43]:
dfx['add_lots'] = res
dfx.lot_quantity = dfx.lot_quantity + dfx.add_lots
dfx.quantity = dfx.lot_quantity * dfx.lot
dfx.sums = dfx.quantity * dfx.price
df_new_port = dfx
df_new_port

,ticker,weights,lot,price,lot_quantity,quantity,sums,add_lots
0,RBCM,21652.9,100,15.710,14,1400,21994.00,0
1,SFIN,20213.4,1,1800.000,31,31,55800.00,5
2,RNFT,13339.4,1,241.500,59,59,14248.50,1
3,AFKS,6511.3,100,24.180,3,300,7254.00,0
5,AFLT,4933.4,10,49.620,10,100,4962.00,0
6,LENT,4102.6,1,965.000,4,4,3860.00,0
7,HHRU,3824.4,1,4500.000,1,1,4500.00,0
8,POSI,3502.0,1,2837.000,1,1,2837.00,0
9,UGLD,3462.4,1000,0.953,4,4000,3812.00,0
11,OZON,3333.3,1,3881.000,1,1,3881.00,0


In [14]:
print("Cумма старого и нового протфеля %d %d" %(df_port.sums.sum(), dfx.sums.sum()))

Cумма старого и нового протфеля 113605 113074


In [57]:
def df_to_dict(dfx):
    d = {row.ticker:row.lot_quantity for _,row in dfx.iterrows()}
#    sd = dict(sorted(d.items()))
    return  d

def sort_dict(d):
    sd = dict(sorted(d.items()))
    return sd
    
old_port = df_to_dict(df_port)
new_port = df_to_dict(df_new_port)
sort_dict(new_port)

{'AFKS': 3,
 'AFLT': 10,
 'ASTR': 3,
 'CIAN': 3,
 'DELI': 9,
 'HHRU': 1,
 'LENT': 4,
 'MOEX': 1,
 'OZON': 1,
 'POSI': 1,
 'RBCM': 14,
 'RNFT': 59,
 'RTKM': 1,
 'RTKMP': 2,
 'SELG': 3,
 'SFIN': 31,
 'UGLD': 4,
 'WUSH': 8}

In [48]:
print("Старый портфель: %d" % df_port.sums.sum())
print("Сумма к распределению: %d" % sum_to_allocate)

print(dfx.sums.sum())


Старый портфель: 113605
Сумма к распределению: 113105
140799.35


In [49]:
def calculate_portfolio_difference(old_portfolio, new_portfolio):
    """
    Рассчитать разницу между двумя портфелями.

    Args:
        old_portfolio: Словарь, где ключом является тикер, а значением - количество акций.
        new_portfolio: Словарь, где ключом является тикер, а значением - количество акций.

    Returns:
        Словарь, где ключом является тикер, а значением - разница между количеством акций в 
        новых и старых портфелях.
    """

    difference = {}
    for ticker in new_portfolio:
        if ticker in old_portfolio:
            difference[ticker] = new_portfolio[ticker] - old_portfolio[ticker]
        else:
            difference[ticker] = new_portfolio[ticker]

    for ticker in old_portfolio:
        if ticker not in new_portfolio:
            difference[ticker] = -old_portfolio[ticker]
    # Сортировка по значению, по возрастанию
    sorted_diff = sorted(difference.items(), key=lambda x: x[1])
    sorted_diff = {k:v for k,v in sorted_diff}
    return difference

In [50]:
rebalance = calculate_portfolio_difference(old_port, new_port)

flag_rebalance = True

In [51]:
rebalance

{'RBCM': 14,
 'SFIN': 31,
 'RNFT': -76,
 'AFKS': 1,
 'AFLT': 2,
 'LENT': -3,
 'HHRU': 0,
 'POSI': 0,
 'UGLD': 4,
 'OZON': 1,
 'DELI': 1,
 'WUSH': 8,
 'SELG': 3,
 'CIAN': 0,
 'ASTR': 3,
 'RTKMP': 2,
 'MOEX': 1,
 'RTKM': 1,
 '0-RUB': -47946,
 'ABIO': -1}

### Split but and sell parts

In [52]:
sell_part = {}
buy_part  = {}
for asset in rebalance:
    qty = rebalance[asset]
    if asset is None  :
        continue
    
    print(asset, qty)

    if asset == "0-RUB":
        continue
    if qty < 0:
        sell_part[asset] = qty
    elif qty >0:
        buy_part[asset] = qty

RBCM 14
SFIN 31
RNFT -76
AFKS 1
AFLT 2
LENT -3
HHRU 0
POSI 0
UGLD 4
OZON 1
DELI 1
WUSH 8
SELG 3
CIAN 0
ASTR 3
RTKMP 2
MOEX 1
RTKM 1
0-RUB -47946
ABIO -1


## Order operation

In [ ]:
account_id = "ebed5b2d-8ff8-4ea7-be10-295f78939cf0"

In [ ]:
from tinkoff.invest import OrderDirection, OrderType
residuals = []
if flag_rebalance:
    with Client(token) as client:

        for asset, qty in rebalance:
            print(asset)
            if asset is None:
                continue
                
            figi = tink.ticker_to_figi(asset, base)
            trading_status = client.market_data.get_trading_status(
                figi=figi
            )
            
            if trading_status.market_order_available_flag and trading_status.api_trade_available_flag:
                if qty < 0:
                    resp = client.orders.post_order(figi=figi,
                                quantity= -qty,
                                direction=OrderDirection.ORDER_DIRECTION_SELL,
                                account_id=account_id,
                                order_type=OrderType.ORDER_TYPE_MARKET,)
                elif qty > 0:
                    resp = client.orders.post_order(figi=figi,
                        quantity=qty,
                        direction=OrderDirection.ORDER_DIRECTION_BUY,
                        account_id=account_id,
                        order_type=OrderType.ORDER_TYPE_MARKET,)
            else:
                print("Не доступно")
                residuals.append((asset, qty))
    flag_rebalance = False

In [ ]:
residuals

## Sandbox accounts

In [61]:
account_id = "ebed5b2d-8ff8-4ea7-be10-295f78939cf0"
logging.basicConfig(format="%(asctime)s %(levelname)s:%(message)s", level=logging.DEBUG)
logger = logging.getLogger(__name__)

### Покупаем портфель

In [62]:
base = tink.get_id_base(token)


portfolio = {'VTBR':10, 'POSI':20, 'RTKM':30, 'IRKT':20, 'RNFT' :100, 'LENT':10, 'CIAN':10, 'ABIO': 10}

for asset in portfolio:
    figi = tink.ticker_to_figi(asset, base)
    print(figi)

2024-04-08 07:35:14,004 INFO:501e4bd322e7239865d2a704b910903b Shares
2024-04-08 07:35:14,693 INFO:9b64e08b2335581b7e3122c7c429f3b6 Currencies
2024-04-08 07:35:14,755 INFO:f33429689e31e022721c5c45986caa19 Futures
2024-04-08 07:35:15,029 INFO:dafe879c4e5fa001130df706b81c183b Bonds
2024-04-08 07:35:15,565 INFO:6fa57b847f97d558951859a7c20e8801 Etfs


BBG004730ZJ9
TCS00A103X66
BBG004S682Z6
BBG000FWGSZ5
BBG00F9XX7H4
BBG0063FKTD9
BBG012YQ6P43
TCS10A0JNAB6


In [ ]:
from tinkoff.invest import OrderDirection, OrderType


with SandboxClient(token) as client:
    for asset in portfolio:
        print(asset)
        figi = tink.ticker_to_figi(asset, base)
        qty = portfolio[asset]
        resp = client.orders.post_order(figi=figi,
                    quantity=qty,
                    direction=OrderDirection.ORDER_DIRECTION_BUY,
                    account_id=account_id,
                    order_type=OrderType.ORDER_TYPE_MARKET,)

### Перебалансировка

In [ ]:
import json

with open('rebalance.json') as f:
    rebalance = json.load(f)

rebalance

### Sell orders

In [ ]:
## Продаем
with SandboxClient(token) as client:

    for asset in rebalance:
        qty = rebalance[asset]
        print(asset, qty)
        if asset is None:
            continue
            
        figi = tink.ticker_to_figi(asset, base)
        trading_status = client.market_data.get_trading_status(
            figi=figi
        )
        
        if trading_status.market_order_available_flag and trading_status.api_trade_available_flag:
            if qty < 0:
                resp = client.orders.post_order(figi=figi,
                            quantity= -qty,
                            direction=OrderDirection.ORDER_DIRECTION_SELL,
                            account_id=account_id,
                            order_type=OrderType.ORDER_TYPE_MARKET,)

In [ ]:
port = tink.TinkPortfolio(SandboxClient, token)
port = port.get_portfolio()
df_port = tink.port_to_df(port, base)
df_port


### Покупаем

In [ ]:
## Покупаем
with SandboxClient(token) as client:

    for asset in rebalance:
        qty = rebalance[asset]
        if asset is None:
            continue
            
        figi = tink.ticker_to_figi(asset, base)
        trading_status = client.market_data.get_trading_status(
            figi=figi
        )
        
        if trading_status.market_order_available_flag and trading_status.api_trade_available_flag:
            if qty > 0:
                print(asset, qty)
                resp = client.orders.post_order(figi=figi,
                            quantity= qty,
                            direction=OrderDirection.ORDER_DIRECTION_BUY,
                            account_id=account_id,
                            order_type=OrderType.ORDER_TYPE_MARKET,)

### Add money to sandbox

In [ ]:
from decimal import Decimal
from tinkoff.invest import MoneyValue
from tinkoff.invest.utils import decimal_to_quotation, quotation_to_decimal

def add_money_sandbox(client, account_id, money, currency="rub"):
    """Function to add money to sandbox account."""
    money = decimal_to_quotation(Decimal(money))
    return client.sandbox.sandbox_pay_in(
        account_id=account_id,
        amount=MoneyValue(units=money.units, nano=money.nano, currency=currency),
    )

with SandboxClient(token) as client:
      # add initial 2 000 000 to sandbox account
        print(add_money_sandbox(client=client, account_id=account_id, money=2000000))
        logger.info(
            "positions: %s", client.operations.get_positions(account_id=account_id)
        )
        print(
            "money: ",
            float(
                quotation_to_decimal(
                    client.operations.get_positions(account_id=account_id).money[0]
                )
            ),
        )

### Create new account

In [ ]:
from tinkoff.invest.sandbox.client import SandboxClient

"""Example - How to set/get balance for sandbox account.
How to get/close all sandbox accounts.
How to open new sandbox account."""
with SandboxClient(token) as client:
    # get all sandbox accounts
    sandbox_accounts = client.users.get_accounts()
    print(sandbox_accounts)

    # close all sandbox accounts
    for sandbox_account in sandbox_accounts.accounts:
        client.sandbox.close_sandbox_account(account_id=sandbox_account.id)

    # open new sandbox account
    sandbox_account = client.sandbox.open_sandbox_account()
    print(sandbox_account.account_id)

    account_id = sandbox_account.account_id